In [13]:
from adata_hf_datasets.file_utils import download_from_link

url = "https://medical-epigenomics.org/papers/schaefer2024/data/datasets/archs4_geo/full_data.h5ad"
save_path = "../data/RNA/raw/geo_full.h5ad"
download_from_link(url, save_path)


Download failed: 403 Client Error: Forbidden for url: https://medical-epigenomics.org/papers/schaefer2024/data/datasets/archs4_geo/full_data.h5ad


False

In [7]:
import anndata

adata = anndata.read_h5ad("../data/RNA/raw/train/geo_7k.h5ad")

In [5]:
# read the file and save a subset of 10k random cells
import scanpy as sc
import numpy as np

adata = sc.read_h5ad(save_path)
adata = adata[np.random.choice(adata.obs.index, 10000, replace=False)]
adata.write_h5ad("../data/RNA/cellxgene_pseudo_bulk_10k.h5ad")